In [1]:
from torcheeg.datasets import CSVFolderDataset
from torcheeg import transforms
from torcheeg.datasets.constants import DEAP_CHANNEL_LOCATION_DICT

In [2]:
dataset = CSVFolderDataset(
    io_path='./temp/muse/data',
    csv_path='./output.csv',
    offline_transform=transforms.Compose([
        transforms.BaselineRemoval(),
        transforms.MeanStdNormalize(),
        transforms.To2d()
    ]),
    online_transform=transforms.ToTensor(),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Binary(5.0),
    ]),
    num_worker=8)

[2025-08-16 01:42:24] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./temp/muse/data.


In [10]:
from torcheeg.model_selection import KFoldGroupbyTrial

k_fold = KFoldGroupbyTrial(n_splits=3,
                           split_path='./temp/muse/split')

In [ ]:
from torch.utils.data import DataLoader
from torcheeg.models import TSCeption

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = TSCeption(num_electrodes=4,
                      num_classes=2,
                      num_T=15,
                      num_S=15,
                      in_channels=1,
                      hid_channels=32,
                      sampling_rate=128,
                      dropout=0.5)

    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=50,
                default_root_dir=f'./temp/muse/model/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')


[2025-08-16 01:48:19] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-08-16 01:48:19] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./temp/muse/split for the next run, if you want to use the same setting for the experiment.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | TSCeption        | 7.8 K  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | MetricCollection | 0      | train
7 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
7.8 K     Train

Epoch 0:   0%|          | 0/3166 [00:00<?, ?it/s] 

RuntimeError: Calculated padded input size per channel: (4 x 82). Kernel size: (14 x 1). Kernel size can't be greater than actual input size